In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pcntoolkit as ptk

import os
import pickle
import pandas as pd
from pcntoolkit.util.hbr_utils import *
import scipy


In [8]:
curdir = os.getcwd()
processing_dirname = "more_flexible_models_lifespan_big"
if not curdir.endswith(processing_dirname):
    processing_dir = os.path.join(curdir,"more_flexible_models_lifespan_big")
    if not os.path.isdir(processing_dir):
        os.mkdir(processing_dir)
    os.chdir(processing_dir)
else:
    curdir=os.path.dirname(processing_dir)
    
print(f"{curdir=}")
print(f"{processing_dir=}")

curdir='/project_cephfs/3022017.02/projects/stijdboe/make_results'
processing_dir='/project_cephfs/3022017.02/projects/stijdboe/make_results/more_flexible_models_lifespan_big'


In [9]:
# Get the lifespan_big data
datadir = '/project_cephfs/3022017.02/projects/stijdboe/Data/10_folds_sexcov/fold_0/'
respfile = os.path.join(datadir, 'Y_train.pkl')       # measurements  (eg cortical thickness) of the training samples (columns: the various features/ROIs, rows: observations or subjects)
covfile = os.path.join(datadir, 'X_train.pkl')        # covariates (eg age) the training samples (columns: covariates, rows: observations or subjects)
testrespfile_path = os.path.join(datadir, 'Y_test.pkl')       # measurements  for the testing samples
testcovfile_path = os.path.join(datadir, 'X_test.pkl')        # covariate file for the testing samples
befile = os.path.join(datadir, 'Z_train.pkl')      # training batch effects file (eg scanner_id, gender)  (columns: the various batch effects, rows: observations or subjects)
testbefile = os.path.join(datadir, 'Z_test.pkl')      # testing batch effects file

output_path = os.path.join(processing_dir, 'Models/')    #  output path, where the models will be written
if not os.path.isdir(output_path):
    os.mkdir(output_path)

log_dir = os.path.join(processing_dir, 'log/')           #
if not os.path.isdir(log_dir):
    os.mkdir(log_dir)


In [10]:
# Parameters shared by all models
likelihood='SHASHb'
model_type='bspline'
linear_mu='True'
linear_sigma='True'
random_intercept_mu='True'
random_intercept_sigma='True'
random_intercept_epsilon='True'
random_intercept_delta='True'

centered_intercept_mu='False'
inscaler_type='standardize'
outscaler_type='standardize'
sampler = 'NUTS'
n_mcmc_samples = 1
n_tuning_samples = 1
n_chains = 1
n_cores = 1
target_accept = 0.99
alg='hbr'
saveoutput='True'
savemodel='True'
binary='True'

In [13]:
# Make a list of features
features = ['Right-Cerebellum-White-Matter',
                'EstimatedTotalIntraCranialVol',
                'Right-Lateral-Ventricle',
                'WM-hypointensities',
                'rh_S_interm_prim-Jensen_thickness', 
                'Brain-Stem']
selectedfeatures = features[2:4]

# Load common data
with open(testcovfile_path,'rb') as file:
    testX = np.array(pickle.load(file))
with open(covfile,'rb') as file:
    X = np.array(pickle.load(file))
with open(testbefile,'rb') as file:
    testZ = np.array(pickle.load(file),dtype=int)
with open(befile,'rb') as file:
    Z = np.array(pickle.load(file),dtype=int)

inscaler_type='standardize'
outscaler_type='standardize'

inscaler = ptk.util.utils.scaler(inscaler_type)
X_train_standardized = inscaler.fit_transform(X)
X_test_standardized = inscaler.transform(testX)

for feature in selectedfeatures:
        
    # For each feature
    # Extract and store the data in a fitting location
    with open(respfile,'rb') as file:
        Y = pickle.load(file)[feature]
    with open(testrespfile_path,'rb') as file:
        testY = pickle.load(file)[feature]
    trainselect = os.path.join(datadir,f'{feature}train.pkl')
    with open(trainselect,'wb') as file:
        pickle.dump(Y, file)
    testselect = os.path.join(datadir,f'{feature}test.pkl')
    with open(testselect,'wb') as file:
        pickle.dump(testY,file)
    Y = np.array(Y)
    testY = np.array(testY)
    outscaler = ptk.util.utils.scaler(outscaler_type)
    Y_train_standardized = np.squeeze(outscaler.fit_transform(Y))
    Y_test_standardized = np.squeeze(outscaler.transform(testY))

    # Make a list of configs
    epsilon_confs = ['False', 'True', 'True']
    delta_confs = ['False', 'False', 'True']
    
#     epsilon_confs = ['True']
#     delta_confs = ['True']
    # For each config
    for c in range(3):
        outputsuffix = f'{feature}_SHASHb_{epsilon_confs[c]}_{delta_confs[c]}_offsets'
        nm = ptk.normative.fit(covfile=covfile,
                               respfile=trainselect,
                               trbefile= befile,
                               log_path=log_dir,
                               saveoutput=saveoutput,
                               output_path=output_path, 
                               savemodel=savemodel,
                               binary=binary,
                               outputsuffix=outputsuffix,
                               alg=alg,
                               sampler=sampler,
                               n_samples=n_mcmc_samples,
                               n_tuning=n_tuning_samples,
                               n_chains=n_chains,
                               cores=n_cores,
                               target_accept=target_accept,
                               inscaler=inscaler_type,
                               outscaler=outscaler_type,
                               likelihood=likelihood,
                               model_type=model_type,
                               linear_mu=linear_mu,
                               random_intercept_mu=random_intercept_mu,
                               centered_intercept_mu=centered_intercept_mu,
                               linear_sigma=linear_sigma,
                               random_intercept_sigma=random_intercept_sigma,
                               linear_epsilon=epsilon_confs[c],
                               random_intercept_epsilon=random_intercept_epsilon,
                               linear_delta = delta_confs[c],
                               random_intercept_delta=random_intercept_delta
                         )
        MAP = nm.hbr.find_map(X_train_standardized, Y_train_standardized, Z)
        with open(os.path.join(processing_dir, f'MAP_{outputsuffix}.pkl'),'wb') as file:
            pickle.dump(MAP, file)
        # Find and store the MAP


Processing data in /project_cephfs/3022017.02/projects/stijdboe/Data/10_folds_sexcov/fold_0/Right-Lateral-Ventricletrain.pkl
Estimating model  1 of 1
slope_mu <class 'pcntoolkit.model.hbr.FixedParameterization'>
slope_mu
dist='normal'
params=(0.0, 10.0)
intercept_mu <class 'pcntoolkit.model.hbr.NonCentralRandomFixedParameterization'>
mu_intercept_mu
dist='normal'
params=(0.0, 10.0)
sigma_intercept_mu
dist='hcauchy'
params=(3.0,)
offset_intercept_mu
dist='normal'
params=(0.0, 1.0)
mu <class 'pcntoolkit.model.hbr.LinearParameterization'>
slope_sigma <class 'pcntoolkit.model.hbr.FixedParameterization'>
slope_sigma
dist='normal'
params=(0.0, 10.0)
intercept_sigma <class 'pcntoolkit.model.hbr.NonCentralRandomFixedParameterization'>
mu_intercept_sigma
dist='normal'
params=(0.0, 1.0)
sigma_intercept_sigma
dist='hcauchy'
params=(1.0,)
offset_intercept_sigma
dist='normal'
params=(0.0, 1.0)
sigma <class 'pcntoolkit.model.hbr.LinearParameterization'>
epsilon <class 'pcntoolkit.model.hbr.FixedPara

Only 1 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [delta, epsilon, offset_intercept_sigma, sigma_intercept_sigma, mu_intercept_sigma, slope_sigma, offset_intercept_mu, sigma_intercept_mu, mu_intercept_mu, slope_mu]


Sampling 1 chain for 1 tune and 1 draw iterations (1 + 1 draws total) took 0 seconds.
/home/preclineu/stijdboe/.conda/envs/stijn1/lib/python3.10/site-packages/pymc3-3.9.3-py3.10.egg/pymc3/sampling.py:628: UserWarning: The number of samples is too small to check convergence reliably.
  warnings.warn(
The chain contains only diverging samples. The model is probably misspecified.


Saving model meta-data...
slope_mu <class 'pcntoolkit.model.hbr.FixedParameterization'>
slope_mu
dist='normal'
params=(0.0, 10.0)
intercept_mu <class 'pcntoolkit.model.hbr.NonCentralRandomFixedParameterization'>
mu_intercept_mu
dist='normal'
params=(0.0, 10.0)
sigma_intercept_mu
dist='hcauchy'
params=(3.0,)
offset_intercept_mu
dist='normal'
params=(0.0, 1.0)
mu <class 'pcntoolkit.model.hbr.LinearParameterization'>
slope_sigma <class 'pcntoolkit.model.hbr.FixedParameterization'>
slope_sigma
dist='normal'
params=(0.0, 10.0)
intercept_sigma <class 'pcntoolkit.model.hbr.NonCentralRandomFixedParameterization'>
mu_intercept_sigma
dist='normal'
params=(0.0, 1.0)
sigma_intercept_sigma
dist='hcauchy'
params=(1.0,)
offset_intercept_sigma
dist='normal'
params=(0.0, 1.0)
sigma <class 'pcntoolkit.model.hbr.LinearParameterization'>
epsilon <class 'pcntoolkit.model.hbr.FixedParameterization'>
epsilon
dist='normal'
params=(0.0, 10.0)
delta <class 'pcntoolkit.model.hbr.FixedParameterization'>
delta
dis


Processing data in /project_cephfs/3022017.02/projects/stijdboe/Data/10_folds_sexcov/fold_0/Right-Lateral-Ventricletrain.pkl
Estimating model  1 of 1
slope_mu <class 'pcntoolkit.model.hbr.FixedParameterization'>
slope_mu
dist='normal'
params=(0.0, 10.0)
intercept_mu <class 'pcntoolkit.model.hbr.NonCentralRandomFixedParameterization'>
mu_intercept_mu
dist='normal'
params=(0.0, 10.0)
sigma_intercept_mu
dist='hcauchy'
params=(3.0,)
offset_intercept_mu
dist='normal'
params=(0.0, 1.0)
mu <class 'pcntoolkit.model.hbr.LinearParameterization'>
slope_sigma <class 'pcntoolkit.model.hbr.FixedParameterization'>
slope_sigma
dist='normal'
params=(0.0, 10.0)
intercept_sigma <class 'pcntoolkit.model.hbr.NonCentralRandomFixedParameterization'>
mu_intercept_sigma
dist='normal'
params=(0.0, 1.0)
sigma_intercept_sigma
dist='hcauchy'
params=(1.0,)
offset_intercept_sigma
dist='normal'
params=(0.0, 1.0)
sigma <class 'pcntoolkit.model.hbr.LinearParameterization'>
slope_epsilon <class 'pcntoolkit.model.hbr.Fi

...

Saving model meta-data...
slope_mu <class 'pcntoolkit.model.hbr.FixedParameterization'>
slope_mu
dist='normal'
params=(0.0, 10.0)
intercept_mu <class 'pcntoolkit.model.hbr.NonCentralRandomFixedParameterization'>
mu_intercept_mu
dist='normal'
params=(0.0, 10.0)
sigma_intercept_mu
dist='hcauchy'
params=(3.0,)
offset_intercept_mu
dist='normal'
params=(0.0, 1.0)
mu <class 'pcntoolkit.model.hbr.LinearParameterization'>
slope_sigma <class 'pcntoolkit.model.hbr.FixedParameterization'>
slope_sigma
dist='normal'
params=(0.0, 10.0)
intercept_sigma <class 'pcntoolkit.model.hbr.NonCentralRandomFixedParameterization'>
mu_intercept_sigma
dist='normal'
params=(0.0, 1.0)
sigma_intercept_sigma
dist='hcauchy'
params=(1.0,)
offset_intercept_sigma
dist='normal'
params=(0.0, 1.0)
sigma <class 'pcntoolkit.model.hbr.LinearParameterization'>
slope_epsilon <class 'pcntoolkit.model.hbr.FixedParameterization'>
slope_epsilon
dist='normal'
params=(0.0, 10.0)
intercept_epsilon <class 'pcntoolkit.model.hbr.NonCentra


Processing data in /project_cephfs/3022017.02/projects/stijdboe/Data/10_folds_sexcov/fold_0/WM-hypointensitiestrain.pkl
Estimating model  1 of 1
slope_mu <class 'pcntoolkit.model.hbr.FixedParameterization'>
slope_mu
dist='normal'
params=(0.0, 10.0)
intercept_mu <class 'pcntoolkit.model.hbr.NonCentralRandomFixedParameterization'>
mu_intercept_mu
dist='normal'
params=(0.0, 10.0)
sigma_intercept_mu
dist='hcauchy'
params=(3.0,)
offset_intercept_mu
dist='normal'
params=(0.0, 1.0)
mu <class 'pcntoolkit.model.hbr.LinearParameterization'>
slope_sigma <class 'pcntoolkit.model.hbr.FixedParameterization'>
slope_sigma
dist='normal'
params=(0.0, 10.0)
intercept_sigma <class 'pcntoolkit.model.hbr.NonCentralRandomFixedParameterization'>
mu_intercept_sigma
dist='normal'
params=(0.0, 1.0)
sigma_intercept_sigma
dist='hcauchy'
params=(1.0,)
offset_intercept_sigma
dist='normal'
params=(0.0, 1.0)
sigma <class 'pcntoolkit.model.hbr.LinearParameterization'>
slope_epsilon <class 'pcntoolkit.model.hbr.FixedPa

Only 1 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [offset_intercept_delta, sigma_intercept_delta, mu_intercept_delta, slope_delta, offset_intercept_epsilon, sigma_intercept_epsilon, mu_intercept_epsilon, slope_epsilon, offset_intercept_sigma, sigma_intercept_sigma, mu_intercept_sigma, slope_sigma, offset_intercept_mu, sigma_intercept_mu, mu_intercept_mu, slope_mu]


Sampling 1 chain for 1 tune and 1 draw iterations (1 + 1 draws total) took 1 seconds.
/home/preclineu/stijdboe/.conda/envs/stijn1/lib/python3.10/site-packages/pymc3-3.9.3-py3.10.egg/pymc3/sampling.py:628: UserWarning: The number of samples is too small to check convergence reliably.
  warnings.warn(
The chain contains only diverging samples. The model is probably misspecified.


Saving model meta-data...
slope_mu <class 'pcntoolkit.model.hbr.FixedParameterization'>
slope_mu
dist='normal'
params=(0.0, 10.0)
intercept_mu <class 'pcntoolkit.model.hbr.NonCentralRandomFixedParameterization'>
mu_intercept_mu
dist='normal'
params=(0.0, 10.0)
sigma_intercept_mu
dist='hcauchy'
params=(3.0,)
offset_intercept_mu
dist='normal'
params=(0.0, 1.0)
mu <class 'pcntoolkit.model.hbr.LinearParameterization'>
slope_sigma <class 'pcntoolkit.model.hbr.FixedParameterization'>
slope_sigma
dist='normal'
params=(0.0, 10.0)
intercept_sigma <class 'pcntoolkit.model.hbr.NonCentralRandomFixedParameterization'>
mu_intercept_sigma
dist='normal'
params=(0.0, 1.0)
sigma_intercept_sigma
dist='hcauchy'
params=(1.0,)
offset_intercept_sigma
dist='normal'
params=(0.0, 1.0)
sigma <class 'pcntoolkit.model.hbr.LinearParameterization'>
slope_epsilon <class 'pcntoolkit.model.hbr.FixedParameterization'>
slope_epsilon
dist='normal'
params=(0.0, 10.0)
intercept_epsilon <class 'pcntoolkit.model.hbr.NonCentra

In [ ]:
with open(os.path.join(processing_dir, f'Models/NM_0_0_lifespanRight-Lateral-VentricleSHASHbTrueTrue.pkl'),'rb') as file:
    model = pickle.load(file)